In [ ]:
import geopandas as gpd
from shapely.geometry import mapping
import rasterio
from rasterio.mask import mask

def create_mask_from_shapefile(shapefile_filepath, corresponding_orthomosaic_filepath,output_folder):

    # open shapefile
    shapefile=gpd.read_file(shapefile_filepath)

    shape=shapefile["geometry"].values
    for i,shp in enumerate(shape):
        zona=shapefile.loc[i,"zona"]
        shapes=[mapping(shp)]
    # with fiona.open(shapefile_filepath, 'r') as shapefile:
    #     # shapes = [feature['geometry'] for feature in shapefile]
    #     for i,feature in enumerate(shapefile):
    #         shapes=[feature["geometry"]]
    # open rasterfile
        with rasterio.open(corresponding_orthomosaic_filepath, 'r') as src:
            out_image, out_transform = mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
            out_meta = src.meta

        out_meta.update({"driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform})

        output_file = output_folder+"/zona_"+str(zona)+".tif"

        with rasterio.open(output_file, "w", **out_meta) as dest:
            dest.write(out_image)

input_raster = r"E:\rastermunicpial\lerma\final.tif"
input_shapefile = r"C:\Users\ASUS\Desktop\DELEGACIONES_EDOMEX\Zona_Lerma.shp"
output=r"E:\rastermunicpial\lerma\zonas_ele"
create_mask_from_shapefile(input_shapefile, input_raster,output)

In [ ]:
import tqdm
import numpy as np
def transform_array2stl(Array:np.array,output_file:str="output.stl"):
  """
  codigo para exportar cualquier array a un archivo stl donde el valor de cada celda representa la altura
  """

  with open(output_file,"w") as f:
          f.writelines("solid s\n")
  f=open(output_file,"a")
  with tqdm.tqdm(total=(Array.shape[0]-1)*(Array.shape[1]-1)) as pbar:
    for j in range(Array.shape[0]-1):
      texto=[]
      for i in range(Array.shape[1]-1):
        if Array[j,i]==0 and Array[j,i+1]==0 and Array[j+1,i+1]==0 and Array[j+1,i]==0:
            pass
        else:
          vertex1=[["vertex",str(j),str(i),str(Array[j,i])],["vertex",str(j),str(i+1),str(Array[j,i+1])],["vertex",str(j+1),str(i+1),str(Array[j+1,i+1])]]
          vertex2=[["vertex",str(j+1),str(i),str(Array[j+1,i])],["vertex",str(j),str(i),str(Array[j,i])],["vertex",str(j+1),str(i+1),str(Array[j+1,i+1])]]
          texto.append("""facet normal 1 1 1\nouter loop\n"""+"\n".join([" ".join(x) for x in vertex1])+"""\nendloop\nendfacet\n"""+"""facet normal 1 1 1\nouter loop\n"""+"\n".join([" ".join(x) for x in vertex2]))
        
        pbar.update(1)
      texto1="\n".join(texto)+"\n"
      f.writelines(texto1)
  f.close()
  # texto="\n".join(texto)+"\n"
  with open(output_file,"a") as f:
          f.writelines("endsolid s")
  

In [ ]:
for i in range(len(zonas)):
    transform_array2stl(r"E:\rastermunicpial\lerma\zonas_ele/zona_"+str(i)+".tif",(r"E:\rastermunicpial\lerma\zonas_ele/zona_3d_"+str(i)+".stl")